In [1]:
# Install Pytorch
%pip install "torch==2.2.2" tensorboard

# Install Hugging Face libraries
%pip install --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"

# Install remaining libraries
%pip install "Pillow==9.5.0" "matplotlib==3.8.0" "numpy==1.24.3" "pandas==2.1.0" "tqdm==4.66.1"


Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.4 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
autotrain-advanced 0.7.87 requires datasets[vision]~=2.19.0, but you have datasets 2.18.0 which is incompatible.
autotrain-advanced 0.7.87 requires packaging==24.0, but you have packaging 23.2 which is incompatible.
autotrain-advanced 0.7.87 requires pandas==2.2.2, but you have pandas 2.1.0 which is incompatible.
autotrain-advanced 0.7.87 requires Pillow==10.3.0, but you have pillow 9.5.0 which is incompatible.
autotrain-advanced 0.7.87 requires protobuf==4.23.4, but you have protobuf 3.20.3 which is incompatible.
autotrain-advanced 0.7.87 requires tiktoken==0.6.0, but you have tiktoken 0.7.0 which is incompatible.
autotrain-advanced 0.7.87 requires tqdm==4.66.2, but you have tqdm 4.66.1 which is incompatible.
autotrain-a


  Obtaining dependency information for Pillow==9.5.0 from https://files.pythonhosted.org/packages/0c/02/7729c8aecbc525b560c7eb283ffa34c6f5a6d0ed6d1339570c65a3e63088/Pillow-9.5.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for matplotlib==3.8.0 from https://files.pythonhosted.org/packages/40/d9/c1784db9db0d484c8e5deeafbaac0d6ed66e165c6eb4a74fb43a5fa947d9/matplotlib-3.8.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for numpy==1.24.3 from https://files.pythonhosted.org/packages/f0/e8/1ea9adebdccaadfc208c7517e09f5145ed5a73069779ff436393085d47a2/numpy-1.24.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pandas==2.1.0 from https://files.pythonhosted.org/packages/b7/f8/32d6b5aa4c4bc045fa2c4c58f88c325facc54721956c6313f0afea8ea853/pandas-2.1.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tqdm==4.66.1 from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8c

In [3]:
import torch
from huggingface_hub import login
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoFeatureExtractor, AutoModelForImageClassification
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from transformers import LlamaForSequenceClassification

In [ ]:
login(api_token = "")

In [ ]:
NLP_model_dir = ""
tokenizer = AutoTokenizer.from_pretrained(NLP_model_dir)
nlp_model = LlamaForSequenceClassification(NLP_model_dir, num_labels=4)
nlp_model.eval()

In [ ]:
model_dir = "E:\\Projekty\\for_work\\docs-det-all-nations\\dit-baseDocument_Classification-ids-seria-21"
model = AutoModelForImageClassification.from_pretrained(model_dir)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_dir)

def cnn_model_function(images_dir):

    image_paths = os.listdir(images_dir)

    document_labels = []

    for path in image_paths:
        image = Image.open(os.path.join(images_dir, path))  
        try:
            inputs = feature_extractor(images=image, return_tensors="pt")
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_label = logits.argmax(-1).item()

            label_mapping = {0: 'id', 1: 'idObcy', 2: 'inne', 3: 'paszport', 4: 'paszportObcy', 5: 'prawoJazdy', 6: 'prawoJazdyObce'}
            predicted_class_name = label_mapping[predicted_label]

            document_labels.append({predicted_class_name: path})  
        
        except Exception as e:  
            print(f"Error processing {path}: {e}")
    
    return document_labels


In [ ]:
def predict_sentiment(text):
    # Force CPU usage
    device = torch.device("cpu")
    nlp_model.to(device)

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Make predictions
    with torch.no_grad():
        outputs = nlp_model(**inputs)
        logits = outputs.logits

    # Get the predicted class
    predicted_class_id = logits.argmax().item()

    return predicted_class_id

In [ ]:
from pdf_ocr import extract_text_from_images

def nlp_model_function(CNN_model_dir, images_dir):
    document_labels = cnn_model_function(CNN_model_dir, images_dir)
    
    values_for_unknown_label = [entry for entry in document_labels if 'inne' in entry]

    nlp_dictionary = {}

    if len(values_for_unknown_label) > 0:
        for entry in values_for_unknown_label:
            path = list(entry.values())[0] #filrs element of every entry which is a one key:value pair dictionary
            text_from_image = extract_text_from_images(path) 
            predicted_class = predict_sentiment(text_from_image)
            nlp_dictionary[path] = predicted_class
    
    return nlp_dictionary

    

In [ ]:
def main_function():
    